In [1]:
import pandas as pd
import sys
import os.path as op 
from numpy.linalg import norm
from gensim.models import FastText
import numpy as np


basename = "/home/adrian/PhD/Tweet-Classification-Diabetes-Distress/"
path_utils = op.join(basename , "utils")
sys.path.insert(0, path_utils)

from sys_utils import load_library
from tweet_utils import tweet_vectorizer

from preprocess import Preprocess
prep = Preprocess()

        
def preprocess_tweet(tweet):
    tweet = prep.replace_contractions(tweet)
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.remove_repeating_characters(tweet)
    tweet = prep.remove_repeating_words(tweet)
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = prep.remove_non_ascii(tweet)
    tweet = prep.replace_numbers(tweet)
    tweet = prep.remove_stopwords(tweet, include_personal_words=False, include_negations=False)
    tweet = [word for word in tweet if word not in ["diabetes", "diabetic"]]
    return tweet

def cosinus_similarity(a, b):
    return np.inner(a,b)/(norm(a)*norm(b))

model = FastText.load("/home/adrian/PhD/Data/FastText_embedding_20190703/ft_wordembeddings_dim300_minCount5_URL-User-toConstant_iter10_20190703")

data = pd.read_parquet("Tweets_groupLabel_20190715.parquet")
data["text_vec"] = data.text.map(lambda tweet: tweet_vectorizer(preprocess_tweet(tweet), model))
print(data.shape)
data.head()

[nltk_data] Downloading package punkt to /home/adrian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/adrian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/adrian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/home/adrian/miniconda3/envs/deepscience/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


no embedding for a8 !!!!!!!!!!!!
no embedding for a8 !!!!!!!!!!!!
no embedding for a8 !!!!!!!!!!!!
(46407, 11)


,id,text,user_name,gender,typeDiabetes,geo_adm1_code,geo_name,user_location,place_full_name,label,text_vec
0,1.129140e+18,When I was initially diagnosed with #type2diab...,Lynda Jimenez,1,1,AZ,Phoenix,"Phoenix, AZ",None,4,"[0.091667205, 0.032091808, -0.081855804, -0.07..."
1,1.102559e+18,Later Twitter it's back to sleep for a nap wok...,Deanna Porter Weick,0,0,MI,Monroe,"Monroe, MI",None,11,"[0.16888438, -0.029450605, 0.0862785, -0.06317..."
2,1.039046e+18,Ugh not ready for this 3hour glucose test tomo...,Esmeralda,2,0,CA,Hemet,Hemet CA,None,22,"[0.07679383, -0.038904045, 0.13049835, -0.2382..."
3,9.581800e+17,@hotfunkytown They also lock up diabetic suppl...,Joyce Pride,0,0,MO,Maryville,"Maryville, Missouri",None,3,"[0.12150481, 0.11272085, 0.11174324, -0.108990..."
4,9.151822e+17,@I_TheeWed “Hey mom! I know you’re dreading th...,Hot Mess Momma🍁,2,0,OK,Oklahoma City,Illinois ✈️ Oklahoma,None,3,"[0.06801716, 0.14217816, 0.018431958, -0.32431..."


In [3]:
topic_labels = {0:"DSMA_support_3", 1: "Diabetes_popstar_Nick_Jonas", 2:"Insulin_(access)", 3:"Negative_feelings_diabetes",
                4:"Diaversary", 5:"Misunderstandings_diabetes", 6:"Living_with_T1D", 7:"Access_insulin_insulinPrice",
                8:"Diagnosis", 9:"Inspiring_relatives_living_with_diabetes_Support", 10:"insulin pump",
                11:"Information_diabetes-rel_events_of_the_day", 12:"Bloodsugar_palette_(beauty products)",
                13:"Inspiring_relatives_living_with_diabetes", 14:"Glucose_guardian", 15:"T1D_hashtags",
                16:"DiabetesAwareness_Support_Donation", 17:"Glucosevariability", 18:"Diabetes_comorbidities",
                19:"Chronic_diseases_diabetes_comorbidities", 20:"DSMA_support_4", 21:"Confusion_between_T1D_and_T2D",
                22:"OGTT", 23:"DSMA_support_2", 24:"Diabetes_care", 25:"DSMA_support_1", 26:"Diabetes_and_entourage_1",
                27:"Life_with_and_without_diabetes", 28:"Diabetes_and_entourage_2", 29:"Insulin_use"}

topic_level1_labels = {
            0:"Diabetes online support", 1:"Miscellenous", 2:"Insulin", 3:"Feelings", 4:"Diabetes", 5:"Feelings", 
            6:"Life with diabetes", 7:"Insulin", 8:"Diabetes", 9:"Entourage", 10:"Insulin", 11:"Diabetes",
            12:"Miscellaneous", 13:"Entourage", 14:"Healthcare", 15:"Diabetes online support", 16:"Diabetes online support",
            17:"Diabetes", 18:"Diab-rel complications", 19:"Diab-rel complications",
            20:"Diabetes online support", 21:"Diabetes", 22:"OGTT", 23:"Diabetes online support",
            24:"Diabetes", 25:"Diabetes online support", 26:"Entourage", 27:"Life with diabetes",
            28:"Entourage", 29:"Insulin"}

In [2]:
regions = list(data.groupby(by="geo_adm1_code")["id"].count().index)



#tuples = list(zip([topic_labels[i] for i in range(0, 30)], [topic_level1_labels[i] for i in range(0, 30)]))
#index = pd.MultiIndex.from_tuples(tuples, names=['Level1_topics', 'topics'])

#print(index)
#print()
#print([topic_labels[i] for i in range(0,30)])
matrix = pd.DataFrame(np.zeros((len(regions), 30)), index=regions, columns=[topic_labels[i] for i in range(0,30)])
#matrix = pd.DataFrame(np.zeros((len(regions), 30)), index=regions, columns=index)
#matrix = pd.DataFrame(np.zeros((len(regions), 30)), index=regions, columns=[str(i) for i in range(0,30)])

for j, group in data.groupby(by="label"):

    for i, v in group.groupby(by="geo_adm1_code")["id"].count().items():

#        matrix.loc[i, str(group.label.values[0])] = v / group.groupby(by="geo_adm1_code")["id"].count().sum()#roup.shape[0]
        matrix.loc[i, topic_labels[group.label.values[0]]] = v / group.groupby(by="geo_adm1_code")["id"].count().sum()#roup.shape[0]
       
print(list(matrix.index))        
matrix.head(5)

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


,DSMA_support_3,Diabetes_popstar_Nick_Jonas,Insulin_(access),Negative_feelings_diabetes,Diaversary,Misunderstandings_diabetes,Living_with_T1D,Access_insulin_insulinPrice,Diagnosis,Inspiring_relatives_living_with_diabetes_Support,...,DSMA_support_4,Confusion_between_T1D_and_T2D,OGTT,DSMA_support_2,Diabetes_care,DSMA_support_1,Diabetes_and_entourage_1,Life_with_and_without_diabetes,Diabetes_and_entourage_2,Insulin_use
AK,0.002656,0.004608,0.000000,0.004390,0.004212,0.002496,0.001343,0.003477,0.001978,0.001313,...,0.003961,0.003348,0.003976,0.000000,0.002234,0.001866,0.000000,0.002451,0.004350,0.002786
AL,0.004648,0.004608,0.000000,0.008780,0.004813,0.004992,0.002686,0.003477,0.005935,0.005909,...,0.005282,0.006696,0.011431,0.000000,0.005212,0.009328,0.011547,0.004902,0.008700,0.006079
AR,0.001992,0.004608,0.004049,0.006829,0.004813,0.004368,0.004701,0.005911,0.000989,0.003940,...,0.005722,0.005580,0.003976,0.000000,0.007446,0.005597,0.004619,0.002451,0.003806,0.004306
AZ,0.019920,0.018433,0.014170,0.028293,0.019856,0.018097,0.026192,0.017385,0.019782,0.020355,...,0.019366,0.021763,0.026839,0.004127,0.022338,0.024254,0.018476,0.017157,0.018488,0.022543
CA,0.130810,0.115207,0.050607,0.116585,0.113718,0.115445,0.111484,0.094576,0.118694,0.128693,...,0.112676,0.122210,0.092445,0.024759,0.114669,0.102612,0.145497,0.122549,0.132681,0.120061


In [17]:
matrix.sum()

DSMA_support_3                                      1.0
Diabetes_popstar_Nick_Jonas                         1.0
Insulin_(access)                                    1.0
Negative_feelings_diabetes                          1.0
Diaversary                                          1.0
Misunderstandings_diabetes                          1.0
Living_with_T1D                                     1.0
Access_insulin_insulinPrice                         1.0
Diagnosis                                           1.0
Inspiring_relatives_living_with_diabetes_Support    1.0
insulin pump                                        1.0
Information_diabetes-rel_events_of_the_day          1.0
Bloodsugar_palette_(beauty products)                1.0
Inspiring_relatives_living_with_diabetes            1.0
Glucose_guardian                                    1.0
T1D_hashtags                                        1.0
DiabetesAwareness_Support_Donation                  1.0
Glucosevariability                              

In [25]:
sociofactors = pd.read_parquet("socioEconom_environmental_factors_20160718.parquet")
print(sociofactors.shape)
sociofactors = sociofactors[sociofactors.LocationAbbr.isin(matrix.index)]
sociofactors["Unemployment-rate-Population-16-years-and-over__2017"] = pd.to_numeric(sociofactors["Unemployment-rate-Population-16-years-and-over__2017"])
sociofactors.sort_values("LocationAbbr", inplace=True)
print(sociofactors.dtypes)
print(sociofactors.shape)
#del sociofactors["LocationAbbr"]
sociofactors.head()

(55, 23)
LocationAbbr                                                                                              object
CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%                   float64
CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%                                 float64
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification    float64
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity                         float64
Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length     float64
HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage                                float64
Mean income (dollars); Estimate; All households                                                          float64
Birth Rate                                                                             

,LocationAbbr,CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%,CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity,Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length,HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage,Mean income (dollars); Estimate; All households,Birth Rate,Fertility Rate,...,Percentage_Diabetes_Diagnosed_2016,Sex_ratio_(males_per_100_females),Estimate_Median_age_years,Percent_One_race_White,Percent_One_race_Black_or_AfricanAmerican,Percent_One_race_American_Indian_and_AlaskaNative,Percent_One_race_Asian,Percent_One_race_NativeHawaiianAndOtherPacificIslander,Percent_One_race_Some_otherRace,Percent_HISPANIC_OR_LATINO
Alaska,AK,25.4,8.9,17.8,19.1,14.4,280.7,92674.0,14.12,71.51,...,7.7,109.7,33.9,65.3,3.2,14.2,6.2,1.2,1.4,738565.0
Alabama,AL,32.1,10.5,16.5,16.3,13.0,437.9,66337.0,12.09,62.05,...,13.2,94.0,38.7,68.4,26.5,0.5,1.3,0.0,1.4,4850771.0
Arkansas,AR,31.9,11.5,15.5,14.4,10.7,421.4,63560.0,12.49,64.98,...,12.1,96.4,37.9,77.3,15.4,0.7,1.4,0.2,2.5,2977944.0
Arizona,AZ,27.0,10.4,15.4,13.3,12.0,268.7,77451.0,11.67,60.84,...,9.7,98.8,37.2,77.5,4.3,4.4,3.1,0.2,7.0,6809946.0
California,CA,19.6,9.3,16.2,16.6,13.3,284.6,101761.0,11.93,58.20,...,9.7,98.7,36.1,60.6,5.8,0.7,14.1,0.4,13.7,38982847.0


In [7]:
for factor in sociofactors.columns:
    print(factor)

CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%
CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification
Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity
Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length
HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage
Mean income (dollars); Estimate; All households
Birth Rate
Fertility Rate
Average Age of Mother
Estimate; Gini Index
Unemployment-rate-Population-16-years-and-over__2017
Percentage_Diabetes_Diagnosed_2016
Sex_ratio_(males_per_100_females)
Estimate_Median_age_years
Percent_One_race_White
Percent_One_race_Black_or_AfricanAmerican
Percent_One_race_American_Indian_and_AlaskaNative
Percent_One_race_Asian
Percent_One_race_NativeHawaiianAndOtherPacificIslander
Percent_One_race_Some_ot

In [26]:
sociofactors = sociofactors[[
    "Mean income (dollars); Estimate; All households",
    "Estimate; Gini Index",
    "Unemployment-rate-Population-16-years-and-over__2017",
    "Sex_ratio_(males_per_100_females)",
    "Estimate_Median_age_years",
    "Percent_One_race_White",
    "Percent_One_race_Black_or_AfricanAmerican",
    "Percent_One_race_American_Indian_and_AlaskaNative",
    "Percent_One_race_Asian",
    "Percent_One_race_NativeHawaiianAndOtherPacificIslander",
    "Percent_One_race_Some_otherRace",
    "Percent_HISPANIC_OR_LATINO"    
]]
sociofactors.head()

,Mean income (dollars); Estimate; All households,Estimate; Gini Index,Unemployment-rate-Population-16-years-and-over__2017,Sex_ratio_(males_per_100_females),Estimate_Median_age_years,Percent_One_race_White,Percent_One_race_Black_or_AfricanAmerican,Percent_One_race_American_Indian_and_AlaskaNative,Percent_One_race_Asian,Percent_One_race_NativeHawaiianAndOtherPacificIslander,Percent_One_race_Some_otherRace,Percent_HISPANIC_OR_LATINO
Alaska,92674.0,0.4180,7.7,109.7,33.9,65.3,3.2,14.2,6.2,1.2,1.4,738565.0
Alabama,66337.0,0.4776,7.4,94.0,38.7,68.4,26.5,0.5,1.3,0.0,1.4,4850771.0
Arkansas,63560.0,0.4732,6.3,96.4,37.9,77.3,15.4,0.7,1.4,0.2,2.5,2977944.0
Arizona,77451.0,0.4690,7.1,98.8,37.2,77.5,4.3,4.4,3.1,0.2,7.0,6809946.0
California,101761.0,0.4889,7.7,98.7,36.1,60.6,5.8,0.7,14.1,0.4,13.7,38982847.0


In [138]:
#Preprocess scale  / normalise values
from sklearn.preprocessing import MinMaxScaler

def normalise(data):
    
    min_max_scaler = MinMaxScaler()
    data["CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%"] /= 100
    data["CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%"] /= 100
    data["Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification"] /= 100
    data["Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity"] /= 100
    data["Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length"] /= 100
    data["HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage"] = min_max_scaler.fit_transform(data["HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage"].values.reshape(-1, 1))
    data["Mean income (dollars); Estimate; All households"] = min_max_scaler.fit_transform(data["Mean income (dollars); Estimate; All households"].values.reshape(-1, 1))
    data["Birth Rate"] /= 100
    data["Fertility Rate"] /= 100
    data["Average Age of Mother"] = min_max_scaler.fit_transform(data["Average Age of Mother"].values.reshape(-1, 1))
    data["Unemployment-rate-Population-16-years-and-over__2017"] /= 100
    data["Percentage_Diabetes_Diagnosed_2016"] /= 100
    
    return data
    
sociofactors_normalised = normalise(sociofactors)
sociofactors_normalised.head()

,CDI__2016__Prevalence_depressive_disorders_adults_>=18years_with_diagnosed_diabetes__%,CDI__2016__Prevalence_diagnosed_diabetes_among_adults_aged_>=18_years__%,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-an-overweight-classification,Obesity-Weight__2014__Percent-of-WIC-children-aged-2-to-4-years-who-have-obesity,Obesity-Weight__2014__Percent-of-WIC-children-aged-3-23-months-old-who-have-a-high-weight-for-length,HeartDiseaseMortality_per100000population__2014__ageAdjusted_3yearAverage,Mean income (dollars); Estimate; All households,Birth Rate,Fertility Rate,Average Age of Mother,Estimate; Gini Index,Unemployment-rate-Population-16-years-and-over__2017,Percentage_Diabetes_Diagnosed_2016
Alaska,0.254,0.089,0.178,0.191,0.144,0.230433,0.538155,0.1412,0.7151,0.4150,0.4180,0.077,0.077
Alabama,0.321,0.105,0.165,0.163,0.130,0.917796,0.110176,0.1209,0.6205,0.1325,0.4776,0.074,0.132
Arkansas,0.319,0.115,0.155,0.144,0.107,0.845649,0.065049,0.1249,0.6498,0.0275,0.4732,0.063,0.121
Arizona,0.270,0.104,0.154,0.133,0.120,0.177962,0.290780,0.1167,0.6084,0.3625,0.4690,0.071,0.097
California,0.196,0.093,0.162,0.166,0.133,0.247486,0.685820,0.1193,0.5820,0.7650,0.4889,0.077,0.097


In [148]:
from sklearn.preprocessing import StandardScaler

sociofactors_standardscaled = StandardScaler().fit_transform(sociofactors)
sociofactors_standardscaled

array([[-4.85706614e-02, -8.40946780e-01,  1.41547630e+00,
         2.18095831e+00,  9.66197413e-01, -8.57772218e-01,
         7.93107212e-01,  1.73794155e+00,  1.79746647e+00,
        -2.37719970e-01, -2.26144466e+00,  1.17191449e+00,
        -1.05174966e+00],
       [ 1.38217029e+00,  1.04335829e-02,  5.01687804e-01,
         9.49966820e-01,  4.43968077e-01,  2.08309822e+00,
        -1.10324886e+00,  1.81461545e-01,  1.72999077e-01,
        -1.39577592e+00,  6.13576122e-01,  9.46659566e-01,
         2.40276767e+00],
       [ 1.33946160e+00,  5.42546309e-01, -2.01226427e-01,
         1.14651168e-01, -4.13980118e-01,  1.77441907e+00,
        -1.30320257e+00,  4.88157113e-01,  6.76137498e-01,
        -1.82620379e+00,  4.01326266e-01,  1.20724859e-01,
         1.71186420e+00],
       [ 2.93098819e-01, -4.27776898e-02, -2.71517850e-01,
        -3.68952630e-01,  7.09471227e-02, -1.08226614e+00,
        -3.03002005e-01, -1.40568802e-01, -3.47816361e-02,
        -4.52933907e-01,  1.98724130e

In [18]:
from sklearn.linear_model import ElasticNetCV
from scipy import stats


def get_summary(model, X, y):

    params = np.append(model.intercept_, model.coef_)
    predictions = model.predict(X)

    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))

    MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))
#    print("newX.shape: ",newX.shape)
#   print("np.dot(newX.T,newX):", np.dot(newX.T,newX).shape)
#    print("MSE:", MSE.shape)
    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b

    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,3)
    params = np.round(params,4)

    return pd.DataFrame({"Coefficients": params, "StandardErrors":sd_b, "t_values":ts_b, "p_values": p_values})


for i in range(0,29):

    #topic = str(i)
    topic = topic_labels[i]
    print("Topic:", topic)
    y = matrix[topic].values
    X = sociofactors.values
    #X = sociofactors_normalised.values
    #X = sociofactors_standardscaled
#    print(y)
#    print("type y:", type(y))
    print("y shape:", y.shape, "   X.shape:", X.shape)
    regr = ElasticNetCV(l1_ratio=[.0001, .001, .01, .1, .5, .7, .9, .95, .99, 1], normalize=True, 
                    cv=3, random_state=0, n_jobs=3, verbose=0, n_alphas=100, max_iter=1000)
    regr.fit(X, y)  

    print("alpha:", regr.alpha_) 
    #print(regr.coef_)
    print("intercept:", regr.intercept_) 
    #print(regr.mse_path_)
    print("l1 ratio:", regr.l1_ratio_)
    print("Score R2:", regr.score(X,y))
    #print(regr.predict([[0, 0]])) 

    new = get_summary(regr, X, y)
    print(new.head(20))
    print()
    print("=============================================")
    print()

Topic: DSMA_support_3
y shape: (51,)    X.shape: (51, 6)
alpha: 0.004264651324388687
intercept: -0.12422160268990994
l1 ratio: 0.1
Score R2: 0.23983188788050303
   Coefficients  StandardErrors  t_values  p_values
0       -0.1242           0.267    -0.465     0.644
1        0.0000           0.000     1.016     0.315
2        0.2798           0.250     1.118     0.269
3        0.0000           0.003     0.000     1.000
4        0.0000           0.002     0.000     1.000
5       -0.0002           0.001    -0.118     0.907
6       -0.0000           0.001    -0.000     1.000


Topic: Diabetes_popstar_Nick_Jonas
y shape: (51,)    X.shape: (51, 6)
alpha: 0.055635304133641705
intercept: -0.03970417764317126
l1 ratio: 0.0001
Score R2: 0.13663073039849283
   Coefficients  StandardErrors  t_values  p_values
0       -0.0397           0.361    -0.110     0.913
1        0.0000           0.000     0.363     0.718
2        0.1519           0.338     0.449     0.655
3        0.0012           0.004     

alpha: 0.03180953551820104
intercept: -0.029570217119327922
l1 ratio: 0.001
Score R2: 0.1689982001144994
   Coefficients  StandardErrors  t_values  p_values
0       -0.0296           0.298    -0.099     0.921
1        0.0000           0.000     0.677     0.502
2        0.1627           0.279     0.583     0.563
3        0.0006           0.003     0.168     0.868
4       -0.0002           0.002    -0.124     0.902
5       -0.0006           0.002    -0.381     0.705
6       -0.0001           0.001    -0.068     0.946


Topic: Glucose_guardian
y shape: (51,)    X.shape: (51, 6)
alpha: 0.0008238207728158645
intercept: -0.13270031017693923
l1 ratio: 1.0
Score R2: 0.14914854190852278
   Coefficients  StandardErrors  t_values  p_values
0       -0.1327           0.351    -0.378     0.707
1        0.0000           0.000     0.000     1.000
2        0.3276           0.329     0.996     0.324
3        0.0000           0.004     0.000     1.000
4        0.0000           0.002     0.000     1.000
5

alpha: 0.004010183480322391
intercept: -0.11088627383962311
l1 ratio: 0.1
Score R2: 0.22111911646990645
   Coefficients  StandardErrors  t_values  p_values
0       -0.1109           0.266    -0.418     0.678
1        0.0000           0.000     0.899     0.373
2        0.2681           0.249     1.078     0.286
3        0.0000           0.003     0.000     1.000
4       -0.0000           0.002    -0.000     1.000
5       -0.0003           0.001    -0.217     0.829
6       -0.0000           0.001    -0.000     1.000


Topic: Diabetes_and_entourage_2
y shape: (51,)    X.shape: (51, 6)
alpha: 0.015151796558550507
intercept: -0.09047626989215184
l1 ratio: 0.01
Score R2: 0.21277807266543125
   Coefficients  StandardErrors  t_values  p_values
0       -0.0905           0.296    -0.306     0.761
1        0.0000           0.000     0.694     0.491
2        0.2536           0.277     0.916     0.364
3        0.0010           0.003     0.297     0.768
4        0.0000           0.002     0.000     

In [150]:
import statsmodels.api as sm
from scipy import stats

for i in range(0,29):

    topic = str(i)
    print("Topic:", i)
    y =  matrix[topic].values
    #X = sociofactors.values
#    X = sociofactors_normalised.values
    X = sociofactors_standardscaled

    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print(est2.summary())
    print()
    print("=================================")
    print()
    print()
    print()

Topic: 0
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.315
Method:                 Least Squares   F-statistic:                     2.773
Date:                Fri, 19 Jul 2019   Prob (F-statistic):            0.00749
Time:                        11:59:07   Log-Likelihood:                 136.50
No. Observations:                  51   AIC:                            -245.0
Df Residuals:                      37   BIC:                            -218.0
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0196      0.003      7.164

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     1.836
Date:                Fri, 19 Jul 2019   Prob (F-statistic):             0.0737
Time:                        11:59:07   Log-Likelihood:                 124.94
No. Observations:                  51   AIC:                            -221.9
Df Residuals:                      37   BIC:                            -194.8
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0196      0.003      5.711      0.0

# Further regression

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lars, LinearRegression, LassoLars, LassoLarsCV
from sklearn.metrics import r2_score, mean_squared_error

for i in range(0,30):

    #topic = str(i)
    topic = topic_labels[i]
    print("Topic:", topic)
    y =  matrix[topic].values
    X = sociofactors.values
    #X = sociofactors_normalised.values
    #X = sociofactors_standardscaled

    print("y shape:", y.shape, "   X.shape:", X.shape)
    #model = LassoLars()
    #regr = GridSearchCV(model, {"alpha":[0.0, 0.01, 0.1, 1.0, 10], "normalize":[True, False], "max_iter":[300, 500, 700]},
    #                    cv=10, n_jobs=3, verbose=1)#, scoring="r2")
    
    model = LinearRegression(n_jobs=3)
    regr = model
#    regr = LassoLarsCV(verbose=1, max_iter=1000, normalize=True, cv=3, n_jobs=3)
    #regr = ElasticNetCV(l1_ratio=[.0001, .001, .01, .1, .5, .7, .9, .95, .99, 1], normalize=True, 
    #                cv=5, random_state=0, n_jobs=3, verbose=0, n_alphas=100, max_iter=1000)
    regr.fit(X, y)  

    #print(dir(regr.best_estimator_))
    #print("Best score: {} \n Best params: {}".format(regr.best_score_, regr.best_params_))
    #print("alpha:", regr.alpha_) 
    #print(regr.coef_)
    #print("intercept:", regr.intercept_) 
    #print(regr.mse_path_)
    #print("l1 ratio:", regr.l1_ratio_)
    print("Score R2:", regr.score(X,y))
    #print(regr.predict([[0, 0]])) 

    new = get_summary(regr, X, y)
#    new = get_summary(regr.best_estimator_, X, y)
    print(new.head(20))
    print()
    print("=============================================")
    print()

Topic: DSMA_support_3
y shape: (51,)    X.shape: (51, 6)
Score R2: 0.3388156098580106
   Coefficients  StandardErrors  t_values  p_values
0       -0.4948           0.249    -1.986     0.053
1        0.0000           0.000     2.186     0.034
2        0.6019           0.233     2.579     0.013
3        0.0016           0.003     0.577     0.566
4        0.0021           0.001     1.450     0.153
5       -0.0005           0.001    -0.360     0.720
6       -0.0006           0.001    -0.618     0.540


Topic: Diabetes_popstar_Nick_Jonas
y shape: (51,)    X.shape: (51, 6)
Score R2: 0.278727317875207
   Coefficients  StandardErrors  t_values  p_values
0       -0.7080           0.330    -2.146     0.037
1        0.0000           0.000     1.070     0.290
2        0.8777           0.309     2.841     0.006
3        0.0012           0.004     0.320     0.750
4        0.0026           0.002     1.381     0.173
5        0.0007           0.002     0.400     0.691
6       -0.0000           0.001   

   Coefficients  StandardErrors  t_values  p_values
0       -0.5258           0.246    -2.141     0.037
1        0.0000           0.000     1.880     0.066
2        0.6913           0.230     3.006     0.004
3        0.0007           0.003     0.271     0.787
4        0.0022           0.001     1.513     0.137
5       -0.0007           0.001    -0.488     0.628
6       -0.0002           0.001    -0.232     0.817




In [23]:
import statsmodels.api as sm
from scipy import stats

for i in range(0,29):
    topic = topic_labels[i]
    print("Topic:", topic)
    y =  matrix[topic].values
    X = sociofactors.values #    X = sociofactors_normalised.values #    X = sociofactors_standardscaled

    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    print(est2.summary())
    print("=================================")
    print()
    print()

Topic: DSMA_support_3
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     3.758
Date:                Thu, 25 Jul 2019   Prob (F-statistic):            0.00418
Time:                        15:16:07   Log-Likelihood:                 129.71
No. Observations:                  51   AIC:                            -245.4
Df Residuals:                      44   BIC:                            -231.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4948      0.2

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     3.483
Date:                Thu, 25 Jul 2019   Prob (F-statistic):            0.00664
Time:                        15:16:07   Log-Likelihood:                 127.82
No. Observations:                  51   AIC:                            -241.6
Df Residuals:                      44   BIC:                            -228.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5420      0.259     -2.096      0.0

# Logistic Regression OR + confidence Interval

In [2]:
#print(data.columns)
#print(sociofactors.columns)
import pandas as pd

states = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


#sociofactors = pd.read_parquet("socioEconom_environmental_factors_20160718.parquet")
#print(sociofactors.shape)
#sociofactors = sociofactors[sociofactors.LocationAbbr.isin(states)]
#sociofactors.sort_values("LocationAbbr", inplace=True)
#sociofactors["geo_adm1_code"] = sociofactors["LocationAbbr"]
#del sociofactors["LocationAbbr"]

#sociofactors = sociofactors[[
#    "Mean income (dollars); Estimate; All households",
#    "Estimate; Gini Index",
#    "Unemployment-rate-Population-16-years-and-over__2017",
#    "Sex_ratio_(males_per_100_females)",
#    "Estimate_Median_age_years",
#    "Percent_One_race_White",
#    "Percent_One_race_Black_or_AfricanAmerican",
#    "Percent_One_race_American_Indian_and_AlaskaNative",
#    "Percent_One_race_Asian",
#    "Percent_One_race_NativeHawaiianAndOtherPacificIslander",
#    "Percent_One_race_Some_otherRace",
#    "Percent_HISPANIC_OR_LATINO", "geo_adm1_code"
#]]


#print(sociofactors.shape)
#print(sociofactors.columns)
#sociofactors.head()


#print(sociofactors.set_index('geo_adm1_code'))
#joined = data.join(sociofactors.set_index('geo_adm1_code'), on="geo_adm1_code")
#joined["Unemployment-rate-Population-16-years-and-over__2017"] = pd.to_numeric(joined["Unemployment-rate-Population-16-years-and-over__2017"])
#print(joined.dtypes)

joined = pd.read_parquet("tweets_sociofactors_joined_citylevel_20190801.parquet")

joined.head()

,mergeColumn,id,text,user_name,gender,typeDiabetes,geo_adm1_code,geo_name,user_location,place_full_name,label,Mean_income_(dollars),Median_age_years,Sex_ratio_malesPer100females,Percent_bachelor's_degree_or_higher,Unemployment_rate_population16yearsAndOver,Gini_index,Percent_Race_aloneOrCombinationWithOtherRaces_White,is_topic,factor_interval
0,AK||Anchorage,9.645795e+17,Omg I just realized my #BloodSugar palette mig...,Amy Nalzy,1,0,AK,Anchorage,Alaska,None,12,105010.0,33.1,104.4,34.6,5.8,0.4192,72.2,0,1
1,AK||Anchorage,9.281538e+17,"@AccessibleMeds Disgusting that, after all thi...",Pibble Lover,2,0,AK,Anchorage,Alaska,None,7,105010.0,33.1,104.4,34.6,5.8,0.4192,72.2,0,1
2,AK||Anchorage,1.138756e+18,@AtomicBlondeDJ Thanks. I found out how very i...,Chris Blankenship,0,2,AK,Anchorage,"Anchorage, AK",None,21,105010.0,33.1,104.4,34.6,5.8,0.4192,72.2,0,1
3,AK||Anchorage,9.933439e+17,"Sometimes--okay, most of the time--I hate diab...",Zach Miller,0,0,AK,Anchorage,Alaska,None,3,105010.0,33.1,104.4,34.6,5.8,0.4192,72.2,0,1
4,AK||Anchorage,1.134380e+18,@RepJayapal My landlord pays $500 dollars on i...,Jon Bauer,0,0,AK,Anchorage,Great State of Alaska,None,7,105010.0,33.1,104.4,34.6,5.8,0.4192,72.2,0,1


In [4]:
joined.columns

Index(['mergeColumn', 'id', 'text', 'user_name', 'gender', 'typeDiabetes',
       'geo_adm1_code', 'geo_name', 'user_location', 'place_full_name',
       'label', 'Mean_income_(dollars)', 'Median_age_years',
       'Sex_ratio_malesPer100females', 'Percent_bachelor's_degree_or_higher',
       'Unemployment_rate_population16yearsAndOver', 'Gini_index',
       'Percent_Race_aloneOrCombinationWithOtherRaces_White', 'is_topic',
       'factor_interval'],
      dtype='object')

In [30]:
import statsmodels.api as sm
import numpy as np


mean_income_denominator = 10000
median_age_years_denominator = 5
unemployment_rate_denominator = 5
Gini_index_denominator = 1
Percent_race_white_denominator = 5
percent_bachelor_denominator = 10


socioFactor_denominator = {"Mean_income_(dollars)":mean_income_denominator, "Median_age_years": median_age_years_denominator,
                           "Unemployment_rate_population16yearsAndOver":unemployment_rate_denominator, "Gini_index":Gini_index_denominator,
                           "Percent_Race_aloneOrCombinationWithOtherRaces_White":Percent_race_white_denominator,
                           "Percent_bachelor's_degree_or_higher":percent_bachelor_denominator, 
                           "gender" : 1, "typeDiabetes": 1}


OR_pd = pd.DataFrame(index=[k+" (in unit {})".format(v) for k,v in socioFactor_denominator.items()])
for i in range(0,30):
    topic = topic_labels[i]
    print("Topic:", topic)
    
    y = joined.label.map(lambda label: 1 if label == i else 0)

    #X = data.geo_adm1_code.map(lambda region: sociofactors[sociofactors.LocationAbbr == region])
#    X = joined[["Mean income (dollars); Estimate; All households",
#                "Estimate; Gini Index",
#                "Unemployment-rate-Population-16-years-and-over__2017",
#                "Sex_ratio_(males_per_100_females)",
#                "Estimate_Median_age_years",
#                "Percent_One_race_White",
#                "Percent_One_race_Black_or_AfricanAmerican",
#                "Percent_One_race_American_Indian_and_AlaskaNative",
#                "Percent_One_race_Asian",
#                "Percent_One_race_NativeHawaiianAndOtherPacificIslander",
#                "Percent_One_race_Some_otherRace",
#                "Percent_HISPANIC_OR_LATINO"]].values
    X = joined[socioFactor_columns]
    X["Mean_income_(dollars)"] = X["Mean_income_(dollars)"] / mean_income_denominator
    X["Median_age_years"] = X["Median_age_years"] / median_age_years_denominator
    X["Unemployment_rate_population16yearsAndOver"] = X["Unemployment_rate_population16yearsAndOver"] / unemployment_rate_denominator
    X["Gini_index"] = X["Gini_index"] / Gini_index_denominator
    X["Percent_Race_aloneOrCombinationWithOtherRaces_White"] = X["Percent_Race_aloneOrCombinationWithOtherRaces_White"] / Percent_race_white_denominator
    X["Percent_bachelor's_degree_or_higher"] = X["Percent_bachelor's_degree_or_higher"] / percent_bachelor_denominator

    
    X = X.values
    X = sm.add_constant(X)
    
    #print("y.shape:", y.shape)
    #print("X.shape:", X.shape)

    res = sm.Logit(y, X).fit()
    #print(res.summary())
    
    params = res.params
    conf = res.conf_int()
    #print(conf)
    conf['OR'] = params
    conf.columns = ['2.5% - OR', '97.5% - OR', 'OR']
    conf.index = ["const"]+socioFactor_columns
    #print("Topic:", topic)
    #print(np.exp(conf))
    conf = np.exp(conf)
    conf["OR [CI]"] = conf.apply(lambda row: "{} [{}; {}]".format(np.round(row['OR'],4), np.round(row["2.5% - OR"], 4), np.round(row['97.5% - OR'],4)), axis=1)
    #print(conf)
    #print()
    #print(conf["OR [CI]"][1::].values)
    
    OR_pd[topic] = conf["OR [CI]"][1::].values

    #break

    print("=================================")
    print()
    #print()

OR_pd.head(10)

Topic: DSMA_support_3


/home/adrian/miniconda3/envs/deepscience/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/adrian/miniconda3/envs/deepscience/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/adrian/miniconda3/envs/deepscience/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

Optimization terminated successfully.
         Current function value: 0.143396
         Iterations 8

Topic: Diabetes_popstar_Nick_Jonas
Optimization terminated successfully.
         Current function value: 0.029629
         Iterations 10

Topic: Insulin_(access)
Optimization terminated successfully.
         Current function value: 0.057861
         Iterations 9

Topic: Negative_feelings_diabetes
Optimization terminated successfully.
         Current function value: 0.181612
         Iterations 7

Topic: Diaversary
Optimization terminated successfully.
         Current function value: 0.150189
         Iterations 8

Topic: Misunderstandings_diabetes
Optimization terminated successfully.
         Current function value: 0.252987
         Iterations 7

Topic: Living_with_T1D
Optimization terminated successfully.
         Current function value: 0.136988
         Iterations 8

Topic: Access_insulin_insulinPrice
Optimization terminated successfully.
         Current function value: 0.23

,DSMA_support_3,Diabetes_popstar_Nick_Jonas,Insulin_(access),Negative_feelings_diabetes,Diaversary,Misunderstandings_diabetes,Living_with_T1D,Access_insulin_insulinPrice,Diagnosis,Inspiring_relatives_living_with_diabetes_Support,...,DSMA_support_4,Confusion_between_T1D_and_T2D,OGTT,DSMA_support_2,Diabetes_care,DSMA_support_1,Diabetes_and_entourage_1,Life_with_and_without_diabetes,Diabetes_and_entourage_2,Insulin_use
Mean_income_(dollars) (in unit 10000),0.9947 [0.9568; 1.034],1.0217 [0.9205; 1.134],0.9657 [0.9079; 1.0272],1.0212 [0.9875; 1.0561],1.0004 [0.9629; 1.0394],1.0222 [0.9966; 1.0484],1.0125 [0.9761; 1.0503],0.9856 [0.9585; 1.0134],1.0266 [0.9818; 1.0734],1.041 [1.0057; 1.0776],...,0.9935 [0.9617; 1.0262],1.02 [0.985; 1.0562],0.9581 [0.9196; 0.9982],0.7255 [0.6932; 0.7594],0.9826 [0.9418; 1.0252],0.9604 [0.8946; 1.0311],1.0321 [0.9628; 1.1063],1.0175 [0.9757; 1.0611],1.0118 [0.9767; 1.0482],0.9894 [0.966; 1.0133]
Median_age_years (in unit 5),0.9978 [0.9322; 1.068],0.8644 [0.7142; 1.0463],0.8902 [0.7876; 1.0062],0.9578 [0.9028; 1.0162],0.9764 [0.9141; 1.0429],1.0235 [0.9756; 1.0736],1.1036 [1.0302; 1.1822],0.9762 [0.9284; 1.0264],0.9767 [0.8997; 1.0604],0.9987 [0.9307; 1.0717],...,1.0448 [0.9885; 1.1044],0.9719 [0.91; 1.0379],0.9161 [0.8639; 0.9715],1.0604 [0.9814; 1.1457],0.9876 [0.9189; 1.0614],0.9362 [0.8334; 1.0518],1.0155 [0.8924; 1.1555],0.9412 [0.8722; 1.0157],1.0886 [1.0252; 1.1559],1.0608 [1.0161; 1.1075]
Unemployment_rate_population16yearsAndOver (in unit 5),1.1598 [1.0099; 1.3319],0.9849 [0.6816; 1.4232],0.7011 [0.5253; 0.9358],1.0566 [0.9383; 1.1898],0.9619 [0.8404; 1.101],0.9917 [0.8963; 1.0971],1.1598 [1.0081; 1.3344],1.0303 [0.9253; 1.1471],1.0132 [0.8522; 1.2045],0.8518 [0.7343; 0.9881],...,1.0606 [0.9467; 1.1883],0.9376 [0.8168; 1.0763],0.8718 [0.7788; 0.9759],0.6804 [0.5883; 0.787],1.0862 [0.938; 1.2579],0.9602 [0.7654; 1.2046],1.0842 [0.8414; 1.3971],1.1088 [0.9506; 1.2935],1.0507 [0.9281; 1.1895],0.9472 [0.862; 1.0407]
Gini_index (in unit 1),0.5167 [0.1305; 2.0461],75.5978 [1.8663; 3062.2294],0.9843 [0.0876; 11.0557],0.9067 [0.278; 2.9569],1.1229 [0.3018; 4.1782],1.6498 [0.6347; 4.288],0.7393 [0.1883; 2.9032],0.7489 [0.2713; 2.0669],1.271 [0.2412; 6.6963],0.4195 [0.1106; 1.591],...,2.0706 [0.6697; 6.4019],1.5128 [0.4145; 5.5217],0.8207 [0.242; 2.7838],0.0016 [0.0004; 0.0064],0.6157 [0.1432; 2.6459],0.1269 [0.0132; 1.2161],5.9806 [0.4727; 75.6698],1.0154 [0.2225; 4.6335],3.138 [0.9082; 10.843],1.4077 [0.5852; 3.3864]
Percent_Race_aloneOrCombinationWithOtherRaces_White (in unit 5),1.0104 [0.9891; 1.0322],1.0064 [0.9516; 1.0644],1.0016 [0.964; 1.0405],1.0062 [0.9879; 1.0248],1.0021 [0.9818; 1.0228],0.9954 [0.9807; 1.0104],0.9885 [0.9677; 1.0098],1.0112 [0.9953; 1.0273],1.0235 [0.9974; 1.0504],0.9591 [0.9391; 0.9795],...,0.9995 [0.9821; 1.0171],1.0023 [0.9823; 1.0228],1.0015 [0.9831; 1.0203],0.8344 [0.8167; 0.8525],1.0059 [0.9834; 1.0289],0.9691 [0.9356; 1.0037],0.9863 [0.9484; 1.0256],1.0134 [0.9899; 1.0374],1.0063 [0.987; 1.0259],0.9961 [0.9824; 1.0099]
Percent_bachelor's_degree_or_higher (in unit 10),1.0599 [0.9852; 1.1403],0.9125 [0.7531; 1.1056],1.2721 [1.1332; 1.428],0.9442 [0.886; 1.0062],0.9399 [0.8753; 1.0093],1.0284 [0.978; 1.0813],1.0334 [0.9598; 1.1127],1.1332 [1.0754; 1.1942],0.9496 [0.8699; 1.0365],1.0078 [0.9388; 1.0819],...,1.005 [0.9456; 1.0681],0.977 [0.9124; 1.0462],0.7662 [0.7145; 0.8215],1.6633 [1.5401; 1.7964],1.0425 [0.9643; 1.1269],0.9973 [0.8797; 1.1305],0.9583 [0.8356; 1.0989],1.0022 [0.925; 1.0857],0.9364 [0.875; 1.0022],1.1201 [1.07; 1.1725]
gender (in unit 1),0.9812 [0.9163; 1.0507],1.279 [1.0707; 1.5279],1.032 [0.9173; 1.1609],1.0653 [1.0045; 1.1298],0.8734 [0.8176; 0.933],0.9955 [0.9492; 1.044],1.092 [1.0193; 1.1699],0.8254 [0.7846; 0.8683],0.9285 [0.8541; 1.0093],0.913 [0.8528; 0.9776],...,0.9785 [0.9254; 1.0348],1.0383 [0.9736; 1.1072],1.4307 [1.3467; 1.5199],1.0824 [1.0082; 1.1621],0.9681 [0.9006; 1.0407],0.9136 [0.8151; 1.0239],1.1123 [0.9815; 1.2605],0.8983 [0.8327; 0.

In [31]:
OR_pd.to_csv("Odd-ratio_topics_over_sociofactors.csv", sep=";")

In [34]:
joined.dtypes

id                                                        float64
text                                                       object
user_name                                                  object
gender                                                      int64
typeDiabetes                                                int64
geo_adm1_code                                              object
geo_name                                                   object
user_location                                              object
place_full_name                                            object
label                                                       int64
text_vec                                                   object
Mean income (dollars); Estimate; All households           float64
Estimate; Gini Index                                      float64
Unemployment-rate-Population-16-years-and-over__2017       object
Sex_ratio_(males_per_100_females)                         float64
Estimate_M